In [1]:
from pathlib import Path
import pickle
import warnings

import gensim
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm

from src.models import algorithms, evaluate_model, lda
from src.utilities import utilities

tqdm.pandas()
warnings.filterwarnings('ignore')

/Users/jensen/Desktop/recommendation-systems/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Load Data

In [2]:
# global variables
DATA_PATH = "data/evaluation"
LDA_PATH = "models/lda"
CATEGORY = "Grocery_and_Gourmet_Food"

# LDA parameters
EPOCHS = 10

# training parameters
N_EPOCHS = 5
LR_ALL = 0.005
BETA = 0.1

# reproducibility checks
SEED = 42
np.random.seed(42)

train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")
lda = pickle.load(open(f"{LDA_PATH}/{CATEGORY}_lda.model", "rb"))

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A23RYWDS884TUL,5.0,This curry paste makes a delicious curry. I j...,2013-05-28,curry paste delicious curry fry chicken vegeta...
1,1,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A945RBQWGZXCK,5.0,I've purchased different curries in the grocer...,2012-09-17,purchase different curry grocery store complet...
2,3,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3AMNY44OP8AOU,4.0,I started a new diet restricting all added sug...,2014-01-23,start new diet restrict added sugar brand suga...
3,4,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A3IB4CQ2QEJLJ8,5.0,So many flavors. I can't begin to tell you how...,2014-04-27,flavor begin tell love mae ploy curry ask reci...
4,5,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",AQA5DF3RWKETQ,5.0,I've used this a lot recently in some of my ch...,2012-11-27,use lot recently chicken dish use lot like spi...
47769,77420,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A192LQZWDYPR4U,5.0,Another quality Reese Peanut Butter Cup produc...,2014-02-27,quality reese peanut butter cup product great ...
47770,77421,B00I33696K,Reese's Miniature Peanut Butter Cups .31oz - 1...,"['Grocery & Gourmet Food', 'Candy & Chocolate'...",A2QKXW3LDQ66P5,5.0,I purchased these for my husband who has every...,2013-02-20,purchase husband love reeses valentine day pre...
47771,77430,B00ID9VSOM,"Viva Labs Organic Coconut Sugar: Non-GMO, Low-...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2P3TGJU301KXD,5.0,this stuff is INCREDIBILY yummy! SO much bette...,2014-07-15,stuff incredibily yummy good regular brown sug...
47772,77456,B00IRL93SY,Barrie House Kenya Estate - AA Single Cup Caps...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AEFE9VDHTQ199,5.0,"Very nice aroma, body and taste! Will buy this...",2014-05-24,nice aroma body taste buy coffee good coffee a...
47773,77508,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2AEZQ3DGBBLPR,2.0,This is a no go for diabetics according to my ...,2014-06-26,diabetic accord wife doctor order intention us...


# Preparing Topic Vectors [Train/Load]

In [4]:
# # generating tokenized reviews
# processed_reviews = train["processedReviewText"].apply(lambda x: x.split())

In [5]:
# # instantiate lda model
# lda_model = lda.LDA(reviews=processed_reviews, n_epochs=EPOCHS)

In [6]:
# %%time
# # training the LDA model
# lda_model.train()

In [7]:
# # save model
# pickle.dump(lda_model, open(MODEL_PATH, "wb"))

# Generating User/Item Topic Vectors

In [8]:
user_idx_map, user_vecs, item_idx_map, item_vecs = utilities.generate_user_item_vectors(train, lda)

In [9]:
# converting factors into numpy obj
user_factors = user_vecs.to_numpy()
item_factors = item_vecs.to_numpy()

In [10]:
# check user factors
user_factors[0,:]

array([0.00133389, 0.00133389, 0.00133389, 0.00133389, 0.12544702,
       0.00133389, 0.00133389, 0.00133389, 0.00133389, 0.00133389,
       0.00133389, 0.00133389, 0.00133389, 0.00133389, 0.00133389,
       0.00133389, 0.0940142 , 0.00133389, 0.00133389, 0.00133389,
       0.00133389, 0.00133389, 0.00133389, 0.00133389, 0.44625637,
       0.00133389, 0.00133389, 0.00133389, 0.00133389, 0.00133389,
       0.00133389, 0.00133389, 0.00133389, 0.00133389, 0.00133389,
       0.00133389, 0.00133389, 0.00133389, 0.00133389, 0.00133389,
       0.00133389, 0.00133389, 0.00133389, 0.2729236 , 0.00133389,
       0.00133389, 0.00133389, 0.00133389, 0.00133389, 0.00133389],
      dtype=float32)

In [11]:
# check item factors
item_factors[0,:]

array([4.4118406e-05, 1.7210656e-01, 4.4118406e-05, 4.4118406e-05,
       5.9069432e-03, 3.3614203e-02, 4.4118406e-05, 4.4118406e-05,
       4.4118406e-05, 4.4118406e-05, 7.5081531e-03, 4.4118406e-05,
       2.0057756e-01, 4.4118406e-05, 1.8977611e-01, 3.8142286e-03,
       1.3531019e-02, 4.4118406e-05, 4.4118406e-05, 4.4118406e-05,
       4.4118406e-05, 4.7609531e-03, 4.4118406e-05, 1.7215887e-02,
       4.4118406e-05, 4.2126946e-02, 4.4118406e-05, 4.4118406e-05,
       2.6400363e-02, 4.4118406e-05, 4.4118406e-05, 4.4118406e-05,
       4.4118406e-05, 4.4118406e-05, 1.8758515e-01, 1.8913865e-02,
       5.0689816e-02, 4.4118406e-05, 4.4118406e-05, 4.4118406e-05,
       4.4118406e-05, 4.4118406e-05, 4.4118406e-05, 4.4118406e-05,
       2.3972191e-02, 4.4118406e-05, 4.4118406e-05, 4.4118406e-05,
       4.4118406e-05, 4.4118406e-05], dtype=float32)

# Generate N-Recommendations = {10, 25, 30, 45}

## Load Test Data

In [12]:
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,2,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1TCSC0YWT82Q0,5.0,I love ethnic foods and to cook them. I recent...,2013-08-03,love ethnic food cook recently purchase produc...
1,8,9742356831,"Mae Ploy Green Curry Paste, 14 oz","['Grocery & Gourmet Food', 'Sauces, Gravies & ...",A1Z7Y2GMAP9SRY,5.0,I like to make my own curry but this is a tast...,2014-06-27,like curry tasty alternative use base kind dif...
2,23,B00004S1C5,"Ateco Food Coloring Kit, 6 colors","['Grocery & Gourmet Food', 'Cooking & Baking',...",A14YSMLYLJEMET,1.0,This product is no where near natural / organi...,2013-03-29,product near natural organic wish review purch...
3,31,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",A2F488C4PLWGEI,5.0,If my wife drinks a cup of this tea when she f...,2014-03-23,wife drink cup tea feel attack come help avoid...
4,32,B00005344V,Traditional Medicinals Organic Breathe Easy Se...,"['Grocery & Gourmet Food', 'Beverages', 'Coffe...",AO1HXV7DWZZIR,5.0,I don't know about the medicinal aspects of th...,2014-02-06,know medicinal aspect tea flavor downright scr...
28001,77519,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A1WT3TVHANP7ZF,3.0,Hmmm. I really wanted to love this sweetener. ...,2014-07-22,hmmm want love sweetener half sugar half stevi...
28002,77520,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A3NEAETOSXDBOM,5.0,"I confess I have a sweet tooth, and love the t...",2014-06-30,confess sweet tooth love taste sugar recognize...
28003,77521,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",AD1ZOPB0BBEHB,4.0,"It has a little of the stevia aftertaste, but ...",2014-07-17,little stevia aftertaste fair compromise able ...
28004,77522,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A18ECVX2RJ7HUE,5.0,i love marinade for grilled flank steak or lon...,2014-05-30,love marinade grilled flank steak london broil...
28005,77523,B00ISVHJ3Y,"Wholesome Sweeteners, Organic Sweet and Lite S...","['Grocery & Gourmet Food', 'Cooking & Baking',...",A2G04D4QZAXL15,3.0,I've been using Truvia (a form of stevia) on m...,2014-05-27,use truvia form stevia cereal greek yogurt yea...


In [13]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))

In [14]:
print(test_user_history)

                  reviewerID  \
0      A00177463W0XWB16A9O05   
1      A022899328A0QROR32DCT   
2      A068255029AHTHDXZURNU   
3      A06944662TFWOKKV4GJKX   
4             A1004703RC79J9   
...                      ...   
13274          AZWRZZAMX90VT   
13275          AZXKAH2DE6C8A   
13276          AZXON596A1VXC   
13277          AZYXC63SS008M   
13278          AZZ5ASC403N74   

                                                    asin  
0                               [B00474OR8G, B00BFM6OAW]  
1                                           [B00CMQDKES]  
2                                           [B001FA1K2G]  
3                                           [B000GFYRHG]  
4                                           [B003GTR8IO]  
...                                                  ...  
13274  [B0007R9L4M, B000CN7BMA, B001EQ5D1K, B002VT3GX...  
13275   [B000MAK41I, B004X8TJP2, B006H34CUS, B007W14RMM]  
13276                           [B001EO5S0I, B00271QQ7Q]  
13277                    

## Preparing Dataset for Surprise's Algorithm

## Instantiate Pre-Initialised Matrix Factorization (Topic Modelling)

In [15]:
# instantiating ti_mf
ti_mf = algorithms.PreInitialisedMF(user_map=user_idx_map,
                                    item_map=item_idx_map,
                                    user_factor=user_factors,
                                    item_factor=item_factors,
                                    learning_rate=LR_ALL,
                                    beta=BETA,
                                    num_epochs=N_EPOCHS,
                                    num_factors=50)

In [16]:
%%time
# fitting to training data
ti_mf.fit(train, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
CPU times: user 2min 6s, sys: 736 ms, total: 2min 7s
Wall time: 2min 8s


In [17]:
%%time
# generate candidate items for user to predict rating
testset = ti_mf.trainset.build_anti_testset()

CPU times: user 32.6 s, sys: 2.02 s, total: 34.6 s
Wall time: 35.6 s


In [18]:
%%time
# predict ratings for all pairs (u, i) that are NOT in the training set
candidate_items = ti_mf.test(testset, verbose=False)

CPU times: user 7min 58s, sys: 2min 3s, total: 10min 2s
Wall time: 11min 20s


## Loop through N = {10, 25, 30, 45}

In [19]:
# generate item popularity
item_popularity = evaluate_model.generate_item_popularity(train)

In [20]:
%%time
n_recommendations = {}
for n in [5, 10, 15, 20]:
    # retrieve the top-n items based on similarities
    top_ns = ti_mf.get_top_n(candidate_items, n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = (evaluate_model.
                           evaluate_recommendations(model_name = 'TI-MF',
                                                    top_ns = top_ns,
                                                    user_rating_history = test_user_history, 
                                                    item_popularity = item_popularity, 
                                                    n = n,
                                                    mf_based = True))
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

The TI-MF has an average recall@5: 0.00762, average novelty@5: 0.88465
The TI-MF has an average recall@10: 0.01075, average novelty@10: 0.90600
The TI-MF has an average recall@15: 0.01380, average novelty@15: 0.91437
The TI-MF has an average recall@20: 0.01621, average novelty@20: 0.92123
CPU times: user 4min 20s, sys: 2min 8s, total: 6min 29s
Wall time: 8min 18s


# Evaluate N-Recommendations

## N=5

In [21]:
top_ns_05 = n_recommendations[5][0]
utilities.retrieve_recommendations(train, top_ns_05, mf_based=True)

For user: A2H84V55USFJQX:
Purchase History:
             asin                                              title
13844  B000ZSZ5S4  Blue Diamond Almonds, Bold Salt &amp; Vinegar,...
41088  B005K4Q1T0  Grove Square Hot Cocoa Dark Chocolate, 24 Sing...

Recommending:

         asin                                              title
0  B000LKTTTW  Imagine Organic Soup, Creamy Tomato, 16 Ounce ...
1  B006IMBHVU     Monster Energy Original, 16 Ounce (Pack of 24)
2  B0014ET2MI  Campbell's Chunky Healthy Request Grilled Chic...
3  B003P7ZO0G  V8 Strawberry Banana, 8 oz. Can (4 packs of 6,...
4  B00014JNI0  YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...


## N=10

In [22]:
top_ns_10 = n_recommendations[10][0]
utilities.retrieve_recommendations(train, top_ns_10, mf_based=True)

For user: A2W9B725TZBXOX:
Purchase History:
             asin                                              title
1645   B0005Z6LLW  Ghirardelli Hot Chocolate Mix , Chocolate Moch...
4683   B000ED9L6C  Bob's Red Mill Raw Shelled Sunflower Seeds (Ke...
8508   B000GATCRQ  Dream Foods International Volcano Lemon Burst,...
15333  B0014EOUAW  V8 V-Fusion Light Peach Mango Juice Drink, 46-...
18192  B001E52ZAS  Post Shredded Wheat Lightly Frosted Cereal, Sp...
21093  B001EPQV1W  Honey Bunches of Oats with Almonds, 14.5-Ounce...
22932  B001G8UC8K  Tootsie Roll Midgees Candy 5 Pound Value Bag 7...
23025  B001GVIRD4                     Grocery &amp; Gourmet Food" />
24945  B001NC8HQS  Idahoan Buttery homestyle flavored mashed pota...
25830  B001SAQ7WQ  Progresso Traditional Soup, Chickarina (Chicke...
26956  B0025UOMY8  Maruchan Yakisoba Spicy Vegetable, 3.79-Ounce ...
33467  B00421DMLM  Libby's Splenda Sliced Peaches, 23.5-Ounce Jar...
34141  B004727CL2  Snack Factory Pretzel Crisps Everything,

## N=15

In [23]:
top_ns_15 = n_recommendations[15][0]
utilities.retrieve_recommendations(train, top_ns_15, mf_based=True)

For user: A1G8ECTOF35O2R:
Purchase History:
             asin                                              title
9115   B000H26J7E  Lindt Excellence Bar, 70% Cocoa Smooth Dark Ch...
47677  B00EKLPLU4              Healthworks Cacao Powder Organic, 1lb

Recommending:

          asin                                              title
0   B00014JNI0  YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...
1   B0001M0Z6Q  Spicy World Peppercorn (Whole)-Black Tellicher...
2   B00DS842HS  Viva Naturals Organic Extra Virgin Coconut Oil...
3   B000EDG4V2       Bob's Red Mill Guar Gum, 8 Ounce (Case of 8)
4   B000Z93FQC               Y.S. Eco Bee Farms Raw Honey - 22 oz
5   B000EDBPO8  Bob's Red Mill White Rice Flour, Organic, 24-O...
6   B001PEWJWC  Garbanzo Beans aka Chickpeas or Ceci Beans | N...
7   B003OGKCDC  Nature's Way Organic Extra Virgin Coconut Oil-...
8   B000F4D5GC  Let's Do Organic Shredded, Unsweetened Coconut...
9   B001O1Q0NA  The Spice Lab Pink Himalayan Salt - 1 Pound X-...
10  B00

## N=20

In [24]:
top_ns_20 = n_recommendations[20][0]
utilities.retrieve_recommendations(train, top_ns_20, mf_based=True)

For user: A1NEKFNO1E743Y:
Purchase History:
            asin                                              title
8695  B000GFYRHG  Bigelow Constant Comment Tea 20-Count Boxes (P...

Recommending:

          asin                                              title
0   B00DS842HS  Viva Naturals Organic Extra Virgin Coconut Oil...
1   B00014JNI0  YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...
2   B000EDG4V2       Bob's Red Mill Guar Gum, 8 Ounce (Case of 8)
3   B000Z93FQC               Y.S. Eco Bee Farms Raw Honey - 22 oz
4   B000EDBPO8  Bob's Red Mill White Rice Flour, Organic, 24-O...
5   B0001M0Z6Q  Spicy World Peppercorn (Whole)-Black Tellicher...
6   B001PEWJWC  Garbanzo Beans aka Chickpeas or Ceci Beans | N...
7   B000F4D5GC  Let's Do Organic Shredded, Unsweetened Coconut...
8   B003OGKCDC  Nature's Way Organic Extra Virgin Coconut Oil-...
9   B000HDK0DC  YumEarth Organic Lollipops, Assorted Flavors, ...
10  B001O1Q0NA  The Spice Lab Pink Himalayan Salt - 1 Pound X-...
11  B0001CXU

# Cross-Analysis for Cold-Start Users (<= 2 Purchased Items)

In [25]:
cold_start_users = utilities.generate_cold_start_users(train)

In [26]:
for n in tuple(zip([5, 10, 15, 20], [top_ns_05, top_ns_10, top_ns_15, top_ns_20])):
    cold_start_top_ns = dict(filter(lambda x: x[0] in cold_start_users, n[1].items()))
    # evaluate how well the recommended items predicted the future purchases
    # on cold start users
    n_recommended_items = (evaluate_model.
                           evaluate_recommendations(model_name = 'TI-MF',
                                                    top_ns = cold_start_top_ns,
                                                    user_rating_history = test_user_history, 
                                                    item_popularity = item_popularity, 
                                                    n = n[0],
                                                    mf_based = True))

The TI-MF has an average recall@5: 0.00823, average novelty@5: 0.88508
The TI-MF has an average recall@10: 0.01177, average novelty@10: 0.90648
The TI-MF has an average recall@15: 0.01505, average novelty@15: 0.91477
The TI-MF has an average recall@20: 0.01753, average novelty@20: 0.92163


# Generating Recommended Items DataFrame

In [27]:
max_recommendations = (utilities
                       .generate_recommendations_df(train,
                                                    n_recommendations,
                                                    "TI-MF",
                                                    mf_based=True,
                                                    max_recommended=20))

In [28]:
max_recommendations

,reviewerID,item_rank,asin,algorithm,title
0,A00177463W0XWB16A9O05,0,B001E5E0D8,TI-MF,"Lavazza Crema e Gusto Ground Coffee Blend, Es..."
1,A00177463W0XWB16A9O05,1,B000EDG4V2,TI-MF,"Bob's Red Mill Guar Gum, 8 Ounce (Case of 8)"
2,A00177463W0XWB16A9O05,2,B001EQ5D6U,TI-MF,"Fiber One Caramel Delight Cereal, 15.6-Ounce B..."
3,A00177463W0XWB16A9O05,3,B00317ES2A,TI-MF,"FiberPlus Antioxidant Bars, Chocolatey Peanut ..."
4,A00177463W0XWB16A9O05,4,B001E6K63A,TI-MF,"Rice Krispies Treats Rice Cereal, 14.2-Ounce ..."
...,...,...,...,...,...
267935,AZZ5ASC403N74,15,B0054TWPNC,TI-MF,"NongShim Bowl Noodle Soup, Hot and Spicy, 3.03..."
267936,AZZ5ASC403N74,16,B000G82L62,TI-MF,"Lundberg Family Farms Wild Blend Rice, 16 Ounc..."
267937,AZZ5ASC403N74,17,B001E5E056,TI-MF,Kashi Heart to Heart Oat Flakes and Blueberry ...
267938,AZZ5ASC403N74,18,B001EO5U3I,TI-MF,"Grocery &amp; Gourmet Food"" />"


# Store in `SQLite` DB

In [29]:
# engine = create_engine("sqlite:///recommender.db", echo=True)

In [30]:
# max_recommendations.to_sql(f"{CATEGORY}", con=engine, if_exists="append")